<a href="https://colab.research.google.com/github/FranciscoAguiar/DataScienceProgramming/blob/master/ProcessingTwitters_UEA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://tiagodemelo.info/datasets/dados-curso-completo.csv.tar.gz
!tar -xvf dados-curso-completo.csv.tar.gz

--2020-10-01 01:24:56--  http://tiagodemelo.info/datasets/dados-curso-completo.csv.tar.gz
Resolving tiagodemelo.info (tiagodemelo.info)... 108.167.188.189
Connecting to tiagodemelo.info (tiagodemelo.info)|108.167.188.189|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143340808 (137M) [application/x-gzip]
Saving to: ‘dados-curso-completo.csv.tar.gz’

dados-curso-complet 100%[===================>] 136.70M  64.1MB/s    in 2.1s    

2020-10-01 01:24:59 (64.1 MB/s) - ‘dados-curso-completo.csv.tar.gz’ saved [143340808/143340808]

dados-curso-completo.csv


In [2]:
import pandas as pd
data = pd.read_csv('dados-curso-completo.csv')
data.head()

,data,usuario,apelido,texto,retweet,seguidores,idioma,lugar,pais,sigla,latitude,longitude,hashtags
0,2020-03-23 18:51:14,Gaboardi,fraslee,@BolsonaroSP Cade o fundão deputado??? Congres...,0,138,pt,NaN,NaN,NaN,NaN,NaN,[]
1,2020-03-18 03:18:14,Seu Zé da Farmácia,seuzedafarmacia,"Em ação preventiva, o Governo do Estado suspen...",0,56,pt,NaN,NaN,NaN,NaN,NaN,[]
2,2020-03-29 12:09:27,Marília Oliveira 🇧🇷,Marioliveira_gb,Incrível como os defensores do fim do distanci...,0,5,pt,NaN,NaN,NaN,NaN,NaN,[]
3,2020-04-01 21:11:07,🚩,santosodeto,Fumaça de cigarro pode propagar coronavírus? h...,0,1885,pt,NaN,NaN,NaN,NaN,NaN,[]
4,2020-03-19 17:49:40,Nova Impressão,novaimpressao_,🚨Todos juntos contra COVID-19💪\n\nSolicite seu...,0,124,pt,Brasília,Brazil,BR,-47.8778,-15.77691,[]


In [3]:
data.describe

<bound method NDFrame.describe of                         data  ...                  hashtags
0        2020-03-23 18:51:14  ...                        []
1        2020-03-18 03:18:14  ...                        []
2        2020-03-29 12:09:27  ...                        []
3        2020-04-01 21:11:07  ...                        []
4        2020-03-19 17:49:40  ...                        []
...                      ...  ...                       ...
1658820  2020-03-13 17:26:18  ...                        []
1658821  2020-05-12 13:02:34  ...                        []
1658822  2020-03-19 18:16:30  ...           ['CORONAVIRUS']
1658823  2020-04-07 18:14:45  ...  ['BolsonaroTemRazaoSim']
1658824  2020-06-02 16:04:51  ...                        []

[1658825 rows x 13 columns]>

In [34]:
total_records = len(data.index)
print('Total records:', total_records)

print('Number of records without date:', data['data'].isnull().sum())
print('Number of records without text:', data['texto'].isnull().sum())

no_city_count = data['lugar'].isnull().sum()
print('Number of records without city:', no_city_count)
print('Percentage of records without city:', (no_city_count/total_records)*100)

print(data['pais'].isnull().sum())
print(data['pais'].isna().sum())
print('lat', data['latitude'].isna().sum())
print('lat', data['latitude'].isnull().sum())
print(data['longitude'].isna().sum())

Total records: 1658825
Number of records without date: 0
Number of records without text: 0
Number of records without city: 1584220
Percentage of records without city: 95.50253944810333
1584220
1584225
1584225
lat 1650105
lat 1650105
1650105


In [5]:
# instalação da biblioteca de tratamento de texto : spacy
!pip install spacy
#!python -m spacy download en_core_web_sm
!python -m spacy download pt

     |████████████████████████████████| 21.2MB 8.8MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-cp36-none-any.whl size=21186282 sha256=9cf8ff0a79f2fc624a2961782f766a1ebc231f8fa47603289da5839858f2d737
  Stored in directory: /tmp/pip-ephem-wheel-cache-xww2euhb/wheels/ea/94/74/ec9be8418e9231b471be5dc7e1b45dd670019a376a6b5bc1c0
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')


In [6]:
import spacy

# Load Portuguese tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("pt")

# Process whole documents
text = ("Caro amigo, A Fumaça de cigarro pode propagar coronavírus?")
doc = nlp(text)

# Analyze syntax
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])
print("Split:", doc.text.split())
print("Split mais coerente: ",[token for token in doc])
print("String ao inves de token", [token.orth_ for token in doc])
print("pontuação: ",[token.orth_ for token in doc if token.is_punct])
print( "Classificacao: ", [(token.orth_, token.pos_) for token in doc])
sentences = [i for i in nlp(text).sents]
print("sentences:", sentences)

# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)

Noun phrases: []
Verbs: ['propagar']
Split: ['Caro', 'amigo,', 'A', 'Fumaça', 'de', 'cigarro', 'pode', 'propagar', 'coronavírus?']
Split mais coerente:  [Caro, amigo, ,, A, Fumaça, de, cigarro, pode, propagar, coronavírus, ?]
String ao inves de token ['Caro', 'amigo', ',', 'A', 'Fumaça', 'de', 'cigarro', 'pode', 'propagar', 'coronavírus', '?']
pontuação:  [',', '?']
Classificacao:  [('Caro', 'PROPN'), ('amigo', 'PROPN'), (',', 'PUNCT'), ('A', 'DET'), ('Fumaça', 'PROPN'), ('de', 'ADP'), ('cigarro', 'NOUN'), ('pode', 'AUX'), ('propagar', 'VERB'), ('coronavírus', 'PROPN'), ('?', 'PUNCT')]
sentences: [Caro amigo, A Fumaça de cigarro pode propagar coronavírus?]
A Fumaça de cigarro MISC


In [35]:
for index, row in data.iterrows():
  doc2 = nlp(row['texto'])
  sentences = [i for i in doc2.sents]
  print("punctuation: ",[token.orth_ for token in doc2 if token.is_punct])
  for sentence in sentences:
    print('sentence:', sentence)

  print('\n')
  if (index == 4):
    break

punctuation:  ['?', '?', '?', '.']
sentence: @BolsonaroSP Cade o fundão deputado??
sentence: ?
sentence: Congresso está sentado em 3BI olhando a pandemia da janela.


punctuation:  [',', '.', '…']
sentence: Em ação preventiva, o Governo do Estado suspendeu as aulas da rede estadual até 31 de março.
sentence: ⁣
⁣

sentence: Esta ação está ali… https://t.co/EPFXOMtwSY


punctuation:  ['…']
sentence: Incrível como os defensores do fim do distanciamento social fazem parecer que as pessoas com alguma doença pré-exis… https://t.co/oEyXDF9cDy


punctuation:  ['?']
sentence: Fumaça de cigarro pode propagar coronavírus?
sentence: https://t.co/JT1oMiUCXf


punctuation:  ['.', '…']
sentence: 🚨
sentence: Todos juntos contra COVID-19💪


sentence: Solicite seu material por e-mail ou WhatsApp.
sentence: Caso venha até a empresa traga seu ma… https://t.co/p6wcoomynZ




# Visão geral dos dados



1. Devem apresentar um resumo (sumário) com as estatísticas dos dados originais, ou seja, sem
qualquer pré-processamento. A apresentação deste tipo de informação relevante para que outras pessoas possam ter uma visão geral dos dados. As estatísticas podem ser apresentadas
através de tabelas e/ou gráficos.



2. As mensagens foram pré-processadas para que as perguntas (questões) fossem identificadas.
Essa coleção de perguntas corresponde ao dataset “DuvidasDB”. A seguir, a equipe deve
apresentar as estatísticas sobre esses dados.



# Temas Discutidos

As equipes devem fazer uma análise sobre os temas que são debatidos nas perguntas dos
postadas pelos usuários. Faça uma análise se as perguntas são relativas aos seguintes temas:



1.   Doença. Quando a pergunta é relativa à doença. Deve-se observar que a doença é identificada por vários nomes. Exemplo: coronavírus, corona, COVID-19, etc.
2.   Medicamento. Quando a pergunta é sobre o uso de determinado medicamento no tratamento
da doença.
3.   Organizações. Quando a pergunta é relativa a uma determinada entidade ou organização.
Emissora de TV, Ministério da Saúde ou empresas, são exemplos de organizações.
4.   Pessoas. Quando a pergunta é sobre determinada pessoa. Por exemplo, a pergunta pode
ser sobre a atuação que determinado político ou pessoa famosa teve durante esse período de
pandemia.





# Visão Temporal

Deve-se fazer uma análise temporal das perguntas que formam o dataset DuvidasDB. Pode-se considerar o intervalo temporal de dias, semanas ou meses. A escolha do intervalo de tempo ficará a cargo das equipes. Exemplos de análise temporal: a) houve um aumento no número de perguntas ao longo do tempo? b) houve uma mudança no perfil das perguntas ao longo do tempo?

# Visão Geográfica

Deve-se fazer uma análise geográfica (espacial) das perguntas que formam o dataset DuvidasDB.
Existem algumas colunas no dataset que trazem a informação das localizações como, por exemplo,
o país, estado e cidade. Em alguns tuítes é possível ainda identificar as coordenadas geográficas de latitude e longitude. Exemplo de análise geográfica: a) os usuários de regiões diferentes fazem perguntas com diferentes focos? Por exemplo, será que os usuários de uma região perguntam mais sobre a doença ou sobre o tratamento? Essa análise ainda pode ser realizada em diversos níveis de
área (cidade, estado ou região). Além de apresentar a distribuição das dúvidas dos usuários por região, a equipe deverá fazer uma análise dessa distribuição. Por exemplo, apresentar as razões(ou hipóteses) da ocorrência dessa distribuição.